In [14]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from itertools import combinations
from sklearn.decomposition import PCA

In [15]:
ph=1.5574E-4
times=[1,2,3,6,7,9,13,16,21,24,31] 
pxt=[(x) for x in list(range(3,50,1))]
neh=range(1,120)

In [16]:
path="C:/Workplace/C++/d2ome_v2/v2/v2/bin/Debug/"
protien="CPSM_MOUSE"  
time=31
peptides=[x.split('_1_')[0] for x in os.listdir(path) if ((".csv" in x) and ("_1_" in x)) ]
peptides=[x  for x in peptides if 'A' in x]
all_AAs=np.unique([c for c in ("".join(sorted([x.replace('_','').replace('2','').replace('3','') for x in peptides])).lower())])
peptidesinfo=dict()
len(peptides)

28

In [17]:
def charcount(pepSeq):
    res={key: 0 for key in ['a', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'p',
       'q', 'r', 's', 't', 'v', 'w', 'y']}     
    for c in pepSeq.lower():
        if c.isalpha():
            res[c]+=1
    return res

In [18]:
for k in range(len(peptides)):
    data=pd.read_csv(f"C:/Workplace/C++/d2ome_v2/v2/v2/bin/Debug/{peptides[k]}_{time}_{protien}.csv")
    data["pxt_neh"]=data.NEH*data.pxt
    data=data[["NEH","pxt","RMSE","pxt_neh","T_NEH","NH"]] 
    minval=min(data.RMSE)
    data=data[data.RMSE==minval]
    
    peptidesinfo[peptides[k]]={"pxt_neh":float(data.pxt_neh),"T_NEH":float(data.T_NEH),"Seq":peptides[k],
                           "charcount":charcount(peptides[k])}



TypeError: can't multiply sequence by non-int of type 'str'

In [ ]:
all_AAs

In [ ]:
visited=set()
visited.add(peptides[0].split('_')[1])
coefficients=[]
for k in range(1,len(peptides)):
    if(peptides[k].split('_')[1] in visited):
        continue
    pep1=peptidesinfo[peptides[0]]
    pep2=peptidesinfo[peptides[k]]
    visited.add(peptides[k].split('_')[1])
    r=pep1["pxt_neh"]/pep2["pxt_neh"]
    coefficients.append([pep1["charcount"][c]-r*pep2["charcount"][c] for c in all_AAs])
    
    if(len(coefficients)==len(all_AAs)-1):
        break
print(len(coefficients))


# visited=set()
# visited.add(peptides[0].split('_')[1])
# coefficients=[]
# for k in range(0,len(peptides)-1,2):
#     if(peptides[k].split('_')[1] in visited):
#         continue
#     pep1=peptidesinfo[peptides[k]]
#     pep2=peptidesinfo[peptides[k+1]]
#     visited.add(peptides[k].split('_')[1])
#     r=pep1["pxt_neh"]/pep2["pxt_neh"]
#     coefficients.append([pep1["charcount"][c]-r*pep2["charcount"][c] for c in all_AAs])
    
#     if(len(coefficients)==len(all_AAs)-1):
#         break
# print(len(coefficients))

In [19]:
if(len(coefficients)<len(all_AAs)-1):
    for k in range(len(peptides)-2,1,-1):
        pep1=peptidesinfo[peptides[len(peptides)-1]]
        pep2=peptidesinfo[peptides[k]]
        r=pep1["pxt_neh"]/pep2["pxt_neh"]
        coefficients.append([pep1["charcount"][c]-r*pep2["charcount"][c] for c in all_AAs])
        if(len(coefficients)==len(all_AAs)-1):
            break


# if(len(coefficients)<len(all_AAs)-1):
#     for k in range(0,len(peptides)-2,3):
# #         if(peptides[k].split('_')[1] in visited):
# #             continue
#         pep1=peptidesinfo[peptides[k]]
#         pep2=peptidesinfo[peptides[k+2]]
#         visited.add(peptides[k].split('_')[1])
#         r=pep1["pxt_neh"]/pep2["pxt_neh"]
#         coefficients.append([pep1["charcount"][c]-r*pep2["charcount"][c] for c in all_AAs])

#         if(len(coefficients)==len(all_AAs)-1):
#             break

In [20]:
coefficients=np.array(coefficients)
coefficients[:,1:].shape

(19, 19)

In [21]:
a_val=4
coefficients=np.array(coefficients)
sol=list(np.linalg.solve(coefficients[:,1:],-a_val*coefficients[:,0])) 
sol.insert(0,a_val)
sol

[4,
 6.190180405214722,
 4.6526549088035285,
 14.920960030842457,
 -13.15599693851137,
 -5.4818000178379815,
 17.499362128763643,
 -10.207782159863337,
 -0.06484012675966994,
 -8.45826130950104,
 -6.088596773116572,
 1.5320777341510197,
 -1.2871297459651299,
 9.838817378497161,
 14.203968681260534,
 1.4581345308825937,
 -8.571090014610693,
 -27.571101327725458,
 -8.175276454992694,
 -11.014360061676953]

In [25]:
# pep1=peptidesinfo[peptides[7]]
# p_neh=0
# for index,c in enumerate(all_AAs):
#     p_neh+=pep1["charcount"][c]*sol[index]
# p_neh

# pep1=peptidesinfo[peptides[1]]
# p_neh=0
# for index,c in enumerate(all_AAs):
    
#     p_neh+=(pep1["charcount"][c]*sol[index])
#     print(c,pep1["charcount"][c],sol[index],pep1["charcount"][c]*sol[index],"\t",p_neh)
# print(p_neh,pep1["T_NEH"])

t_neh,e_neh=[],[]
for i in range (len(peptides)):
    pep1=peptidesinfo[peptides[i]]
    p_neh=0
    for index,c in enumerate(all_AAs):
#         print(c,pep1["charcount"][c],pep1["charcount"][c]*sol[index])
        p_neh+=(pep1["charcount"][c]*sol[index])
#     print("\n",p_neh,"\t",pep1["T_NEH"])
    t_neh.append(pep1["T_NEH"])
    e_neh.append(p_neh)
    
res=pd.DataFrame([peptides,t_neh,e_neh])
res=res.transpose()
res.columns=["peptides","True_NEH","e_NEH"]
res["dif"]=res.True_NEH-res.e_NEH
print(np.mean(res.dif),np.median(res.dif),np.std(res.dif))
res.sort_values(by='dif')

KeyError: '_AFAmTNQILVER_2'

In [23]:
temp=pd.DataFrame(coefficients)
temp.columns=all_AAs
temp 

,a,c,d,e,f,g,h,i,k,l,m,n,p,q,r,s,t,v,w,y
0,-1.186915,1.000000,1.000000,-0.093458,1.000000,-0.093458,-2.093458,1.000000,-2.093458,2.000000,1.000000,0.000000,0.000000,0.000000,1.000000,-1.093458,-4.186915,0.000000,-2.093458,-1.093458
1,1.052174,-0.947826,-0.947826,2.000000,1.000000,-1.895652,-1.947826,-0.947826,0.000000,-3.843477,1.000000,0.000000,0.000000,0.000000,-0.947826,1.000000,0.000000,0.000000,0.000000,1.000000
2,1.386555,1.000000,-0.613445,-1.226890,1.000000,2.000000,0.000000,1.000000,0.000000,0.386555,1.000000,0.000000,0.000000,0.000000,-0.613445,-2.226890,-1.613445,-1.613445,0.000000,-0.613445
3,-0.027027,1.000000,-1.018018,-0.018018,-0.009009,2.000000,0.000000,-0.009009,0.000000,2.000000,1.000000,0.000000,0.000000,-1.009009,-0.009009,-0.009009,-3.027027,-1.009009,0.000000,1.000000
4,0.610667,1.000000,-0.194667,-0.389333,1.000000,2.000000,0.000000,-1.389333,-1.194667,-1.584000,1.000000,0.000000,-1.194667,0.000000,-0.194667,-0.194667,0.000000,0.000000,0.000000,-0.194667
5,1.135923,1.000000,1.000000,0.135923,-0.864077,0.135923,0.000000,-0.864077,-1.864077,2.000000,-0.864077,-1.864077,-3.728155,0.000000,-0.864077,-2.728155,0.000000,0.000000,0.000000,1.000000
6,2.056842,1.000000,-0.886316,0.113684,0.056842,1.056842,-0.943158,0.056842,-0.943158,-0.829474,1.000000,-0.943158,0.000000,-1.886316,1.000000,1.000000,0.000000,-0.943158,0.000000,1.000000
7,1.712644,1.000000,-0.287356,-0.574712,-0.287356,2.000000,-1.287356,-0.287356,-1.287356,-3.149425,1.000000,0.000000,0.000000,-1.287356,1.000000,1.000000,-1.287356,0.000000,0.000000,1.000000
8,1.880000,1.000000,-2.360000,-0.240000,1.000000,2.000000,0.000000,-0.120000,-1.120000,0.880000,-0.120000,-1.120000,-2.240000,0.000000,-0.120000,1.000000,-1.120000,-1.120000,0.000000,1.000000
9,-1.800000,1.000000,1.000000,2.000000,-0.200000,0.800000,0.000000,-1.400000,-1.200000,2.000000,1.000000,-2.400000,-1.200000,-3.600000,1.000000,1.000000,0.000000,0.000000,-1.200000,1.000000


In [24]:
for c in range( len(coefficients)):
    print(np.dot(coefficients[c],sol))

-6.65318778941655e-15
1.4210854715202004e-14
-1.0873225873816648e-14
0.0
-8.147913081835126e-15
0.0
-3.552713678800501e-15
-5.329070518200751e-15
-1.0658141036401503e-14
1.5987211554602254e-14
5.329070518200751e-15
1.2434497875801753e-14
1.0709882866243182e-16
-6.5056576968527334e-15
2.2474261470882476e-15
-1.7858803030240576e-14
-3.432810914345198e-15
-9.866832342074053e-15
-1.0658141036401503e-14
